<h1 align="center"> Deep Learning gender from name - RNN LSTMs </h1>

#### we will use an LSTM RNN to learn gender as f(name). we will use a stacked LSTM with many-to-one architecture feeding charecter inputs and predicting a binary outcome M/F. loss function used will be binary_crossentropy (a special case of categorical_crossentropy with m=2) and using adam optimizer (modified SGD) sample input /output would like this <br> ['r','a','k','e','s','h',' '] - male<br> ['p','r','a','d','e','e','p'] - male<br> ['g','a','n','g','a',' '] - female<br> and so on...

<img src="LSTM_RNN_architecture.jpg" width="800" height="600"/>

In [1]:
from __future__ import print_function

from sklearn.preprocessing import OneHotEncoder
from keras.layers.core import Dense, Activation, Dropout
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb
import pandas as pd
import numpy as np
import os

Using TensorFlow backend.
/Users/DuaaTashkandi/anaconda/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
#parameters
maxlen = 30 # FIXME 
labels = 2

In [3]:
input = pd.read_csv("data_new/merged.csv",header=None)
input.columns = ['first', 'last','b_or_n']

# remove encode 
input['first'] = input['first'].str[2:]
input['first'] = input['first'].str[:-1]
input['last'] = input['last'].str[2:]
input['last'] = input['last'].str[:-1]

#print(type(b'usdgfuia'.decode('utf-8')))

input['firstlen']= [len(str(i)) for i in input['first']]
input1 = input[(input['firstlen'] >= 2) ] #FIXME
print(input1)

                       first              last  b_or_n  firstlen
0             ithalohfonseca        chaverinho       1        14
1                    gustavo  gon\xc3\xa7alves       1         7
2                     rafael           geraldo       1         6
3                      anton          prasetyo       1         5
4                   cristyan            victor       1         8
5      valsemorborgesdesouza              neto       1        21
6            diegodereknobre             nobre       1        15
7                 paulonatan            santos       1        10
8              jardelurgalde          oliveira       1        13
9                   deolindo           barbosa       1         8
10         gabrielbitencourt         figueredo       1        17
11                   eduardo          carvalho       1         7
12                   matheus               rct       1         7
13                  fernando             cesar       1         8
14                   gust

In [4]:
input1.groupby('b_or_n')['first'].count() #FIXMEBEN

b_or_n
0    30000
1    29996
Name: first, dtype: int64

In [5]:
firsts = input['first']
#print(firsts)
labels = input['b_or_n']
vocab = set(' '.join([str(i) for i in firsts]))
vocab.add('END')
len_vocab = len(vocab)

In [6]:
print(vocab)
print("vocab length is ",len_vocab)
print ("length of input is ",len(input1))

{'n', 'j', 'o', '2', 'i', ' ', 'e', 'l', '4', '6', 'END', 'p', '7', 'h', 'f', 'q', 'k', '5', '3', 'b', '0', 't', 's', '8', 'x', 'r', 'z', 'w', 'c', 'm', '\\', '9', 'v', 'y', 'u', '1', 'a', 'd', 'g'}
vocab length is  39
length of input is  59996


In [7]:
char_index = dict((c, i) for i, c in enumerate(vocab))

In [8]:
print(char_index)

{'n': 0, 'j': 1, 'o': 2, '2': 3, 'i': 4, ' ': 5, 'e': 6, 'l': 7, '4': 8, '6': 9, 'END': 10, 'p': 11, '7': 12, 'h': 13, 'f': 14, 'q': 15, 'k': 16, '5': 17, '3': 18, 'b': 19, '0': 20, 't': 21, 's': 22, '8': 23, 'x': 24, 'r': 25, 'z': 26, 'w': 27, 'c': 28, 'm': 29, '\\': 30, '9': 31, 'v': 32, 'y': 33, 'u': 34, '1': 35, 'a': 36, 'd': 37, 'g': 38}


In [9]:
#train test split 
msk = np.random.rand(len(input1)) < 0.8

# we dont want to split 
#msk = [True]*len(input1)

train = input1[msk]
test = input1[~msk]

print(train)

                   first              last  b_or_n  firstlen
1                gustavo  gon\xc3\xa7alves       1         7
2                 rafael           geraldo       1         6
3                  anton          prasetyo       1         5
7             paulonatan            santos       1        10
8          jardelurgalde          oliveira       1        13
10     gabrielbitencourt         figueredo       1        17
12               matheus               rct       1         7
13              fernando             cesar       1         8
14               gustavo           batalha       1         7
15               muhamad           riyandi       1         7
16      oohtalldomatheus               pjl       1        16
17               gabriel     ara\xc3\xbajo       1         7
19                thiago          urameshi       1         6
20             juliandra         bimantara       1         9
21               gustavo            araujo       1         7
22             guilherme

In [10]:
# take input upto max and truncate rest
# encode to vector space(one hot encoding)
# padd 'END' to shorter sequences
train_X = []
trunc_train_first = [str(i)[0:30] for i in train['first']]
for i in trunc_train_first:
    tmp = [char_index[j] for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(char_index["END"])
    train_X.append(tmp)

In [11]:
np.asarray(train_X).shape

(47840, 30)

In [12]:
def set_flag(i):
    tmp = np.zeros(39);
    tmp[i] = 1
    return(tmp)

In [13]:
set_flag(3)

array([ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

#### modify the code above to also convert each index to one-hot encoded representation

In [14]:
#take input upto max and truncate rest
#encode to vector space(one hot encoding)
#padd 'END' to shorter sequences
#also convert each index to one-hot encoding
train_X = []
train_Y = []
trunc_train_first = [str(i)[0:maxlen] for i in train['first']]
for i in trunc_train_first:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    train_X.append(tmp)
for i in train['b_or_n']:
    if i == 1:
        train_Y.append([1,0])
    else:
        train_Y.append([0,1])
    

In [15]:
np.asarray(train_X).shape

(47840, 30, 39)

In [16]:
np.asarray(train_Y).shape

(47840, 2)

#### build model in keras ( a stacked LSTM model with many-to-one arch ) here 30 sequence and 2 output each for one category(m/f)

In [17]:
#build the model: 2 stacked LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(512, return_sequences=True, input_shape=(maxlen,len_vocab)))
model.add(Dropout(0.2))
model.add(LSTM(512, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(2))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

Build model...


In [18]:
test_X = []
test_Y = []
trunc_test_first = [str(i)[0:maxlen] for i in test['first']]
for i in trunc_test_first:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    test_X.append(tmp)
for i in test['b_or_n']:
    if i == 1:
        test_Y.append([1,0])
    else:
        test_Y.append([0,1])
    

In [19]:
print(np.asarray(test_X).shape)
print(np.asarray(test_Y).shape)

(12156, 30, 39)
(12156, 2)


In [ ]:
batch_size=1000
model.fit(train_X, train_Y,batch_size=batch_size,nb_epoch=10,validation_data=(test_X, test_Y))

/Users/DuaaTashkandi/anaconda/lib/python3.6/site-packages/keras/models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 47840 samples, validate on 12156 samples
Epoch 1/10
47840/47840 [==============================] - 813s 17ms/step - loss: 0.6112 - acc: 0.6554 - val_loss: 0.5452 - val_acc: 0.7470
Epoch 2/10
47000/47840 [============================>.] - ETA: 13s - loss: 0.5431 - acc: 0.7449

In [ ]:
score, acc = model.evaluate(test_X, test_Y)
print('Test score:', score)
print('Test accuracy:', acc)

In [288]:
name=["sandhya","jaspreet","rajesh"]
X=[]
trunc_name = [i[0:maxlen] for i in name]
for i in trunc_name:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    X.append(tmp)
pred=model.predict(np.asarray(X))

In [289]:
pred

array([[ 0.62356585,  0.37643418],
       [ 0.72094178,  0.27905828],
       [ 0.90337974,  0.09662029]], dtype=float32)

#### Lets train more, clearly some very simple female names it doesnt get right like mentioned above (inspite it exists in training data)

In [290]:
batch_size=1000
model.fit(train_X, train_Y,batch_size=batch_size,nb_epoch=50,validation_data=(test_X, test_Y))

Train on 12198 samples, validate on 3028 samples
Epoch 1/50
12198/12198 [==============================] - 145s - loss: 0.4107 - acc: 0.8137 - val_loss: 0.4408 - val_acc: 0.7966
Epoch 2/50
12198/12198 [==============================] - 144s - loss: 0.3912 - acc: 0.8254 - val_loss: 0.4479 - val_acc: 0.7936
Epoch 3/50
12198/12198 [==============================] - 145s - loss: 0.3927 - acc: 0.8228 - val_loss: 0.4511 - val_acc: 0.7982
Epoch 4/50
12198/12198 [==============================] - 145s - loss: 0.3730 - acc: 0.8344 - val_loss: 0.4253 - val_acc: 0.8071
Epoch 5/50
12198/12198 [==============================] - 145s - loss: 0.3640 - acc: 0.8396 - val_loss: 0.4240 - val_acc: 0.8164
Epoch 6/50
12198/12198 [==============================] - 145s - loss: 0.3490 - acc: 0.8505 - val_loss: 0.4183 - val_acc: 0.8180
Epoch 7/50
12198/12198 [==============================] - 145s - loss: 0.3411 - acc: 0.8542 - val_loss: 0.4089 - val_acc: 0.8243
Epoch 8/50
12198/12198 [========================

In [460]:
score, acc = model.evaluate(test_X, test_Y)
print('Test score:', score)
print('Test accuracy:', acc)

3028/3028 [==============================] - 16s    
Test score: 0.448404541104
Test accuracy: 0.864266842879


<h3 align="center"> lets look at the loss and accuracy chart as a function of epochs </h3><img src="loss_charts.bmp" alt="loss charts" width="500" height="350"/><img src="acc_charts.bmp" alt="loss charts"  width="500" height="350"/>

In [342]:
name=["sandhya","jaspreet","rajesh","kaveri","aditi deepak","arihant","sasikala","aditi","ragini rajaram"]
X=[]
trunc_name = [i[0:maxlen] for i in name]
for i in trunc_name:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    X.append(tmp)
pred=model.predict(np.asarray(X))
pred

array([[ 0.0859881 ,  0.91401184],
       [ 0.96310365,  0.03689628],
       [ 0.7148453 ,  0.28515476],
       [ 0.02246205,  0.97753793],
       [ 0.13607673,  0.86392319],
       [ 0.99559009,  0.00440993],
       [ 0.05380283,  0.94619709],
       [ 0.55060732,  0.44939268],
       [ 0.10676169,  0.89323831]], dtype=float32)

In [345]:
name=["abhi","abhi deepak","mr. abhi"]
X=[]
trunc_name = [i[0:maxlen] for i in name]
for i in trunc_name:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    X.append(tmp)
pred=model.predict(np.asarray(X))
pred

array([[ 0.15557961,  0.84442037],
       [ 0.25342518,  0.74657482],
       [ 0.8618474 ,  0.13815261]], dtype=float32)

In [502]:
name=["rajini","rajinikanth","mr. rajini"]
X=[]
trunc_name = [i[0:maxlen] for i in name]
for i in trunc_name:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    X.append(tmp)
pred=model.predict(np.asarray(X))
pred

array([[ 0.33718896,  0.66281104],
       [ 0.99896383,  0.00103616],
       [ 0.99664474,  0.00335527]], dtype=float32)

In [450]:
#save our model and data
model.save_weights('gender_model',overwrite=True)
train.to_csv("train_split.csv")
test.to_csv("test_split.csv")

In [464]:
evals = model.predict(test_X)
prob_m = [i[0] for i in evals]

In [479]:
out = pd.DataFrame(prob_m)
out['name'] = test.name.reset_index()['name']
out['m_or_f']=test.m_or_f.reset_index()['m_or_f']

In [483]:
out.head(10)
out.columns = ['prob_m','name','actual']
out.head(10)
out.to_csv("gender_pred_out.csv")